# Projeto Banco de Dados I
### Professor Rogério Mainardes
### Alunos Erick Henrique e Paulo Vitor 

Esse projeto consiste na análise descritiva e de 10 perguntas que serão respondidas através do uso de SQL para um dataset.
O dataset escolhido está disponível no [github](https://github.com/ValdisW/datasets/blob/master/video-game-sales.csv).

Os dados escolhidos mostram as vendas de jogos de videogame lançados entre 1980 e 2020 que tiveram mais de 100.000 vendas nas regiões América do Norte, União Europeia, Japão ou outros (resto do mundo).

In [1]:
-- Criação da tabela videogame_sales no postgres
CREATE TABLE videogame_sales(
    ranking integer,
    game_name varchar(200),
    platform varchar(20),
    release_year smallint,
    genre varchar(20),
    publisher varchar(50),
    na_sales real,
    eu_sales real,
    jp_sales real,
    other_sales real,
    global_sales real
);

In [ ]:
-- cópia dos dados do arquivo videogame-sales para o postgres
COPY videogame_sales 
FROM 'C:\video-game-sales.csv'
WITH(
    FORMAT csv,
    DELIMITER ',', 
    HEADER,
    NULL 'N/A'
);